In [1]:
!pip install pandas openpyxl

In [2]:
import pandas as pd
import re
import os

In [3]:
!pip install pandas openpyxl

In [4]:
import pandas as pd
import re

In [19]:
for category, kws in keywords.items():
    df_k = pd.DataFrame({'keyword': kws})
    df_k.to_csv(f'{category}.csv', index=False)

with pd.ExcelWriter('ukwanita_lexicon.xlsx') as writer:
    for category, kws in keywords.items():
        pd.DataFrame({'keyword': kws}).to_excel(writer, sheet_name=category, index=False)

In [6]:
from google.colab import files
uploaded = files.upload()

Saving final dataset.xlsx to final dataset.xlsx


In [7]:
filename = list(uploaded.keys())[0]
df = pd.read_excel(filename)

In [8]:
keywords = {
    "UK_victim": [
        "korban", "pikmi", "salah cwe", "salah cewe", "nyalahin",
        "baju", "pakaian", "terbuka", "emang mau", "kenapa gak nolak",
        "ngapain keluar malem", "harusnya diem", "salah sendiri",
        "wajar digituin", "pantes dilecehkan"
    ],
    "UK_patriarki": [
        "patriarki", "feminis", "cwe masak", "cewek di dapur",
        "cewek gak cocok kerja", "wanita harus nurut", "istri harus taat",
        "perempuan ga usah sekolah", "cewek tempatnya di rumah",
        "bisa masak", "harus bisa masak", "cewek harus masak"
    ],
    "UK_misogyny": [
        "goblok", "tolol", "matre", "murahan", "kuliah", "dapur", "betina",
        "nyusahin", "perawan tua", "gatel", "ga laku", "siapa suruh", "cewe hamil",
        "cewek bodoh", "ngapain jadi cewek", "bikin ribet", "cewek brengsek",
        "brengsek lu cewek", "susah ngertiin cewek", "cewek gak berguna", "manja banget"
    ],
    "UK_mockery": [
        "pikmi", "sensi mba", "udah jelek", "cewe gatau diri", "cwe gatau diri",
        "hidup susah", "takut hidup susah", "sok kuat", "bucin banget",
        "cuma modal cantik", "nangis mulu", "mau enaknya doang", "cewek manja",
        "gak bisa mikir", "sok feminis", "pura-pura kuat", "cewek keras kepala",
        "gatau diri", "sensi amat", "sensi"
    ],
    "Non_UK": [
        "baik hati", "baik cantik", "cantik nurut", "imut cantik", "masya allah",
        "tabarakallah", "anak baik", "anak cantik", "baik kamu",
        "kamu cantik", "baik banget", "cantik banget", "cantik luar dalam", "baik luar dalam",
        "wanita baik", "perempuan cerdas", "perempuan sukses", "wanita kuat", "cewek rajin",
        "perempuan hebat", "support wanita", "cewek baik",
        "cewek tangguh", "wanita tangguh", "doa terbaik", "semangat terus", "selalu kuat",
        "hebat banget", "salut sama cewek", "terima kasih", "kamu hebat", "tetap semangat",
        "wanita mandiri", "wanita sabar", "cewek mandiri",
        "cewek keren", "wanita keren"
    ]
}

In [9]:
df = df.iloc[:, [1, 3, 14]].copy()
df.columns = ['created_at', 'full_text', 'username']
df

,created_at,full_text,username
0,Tue Apr 01 03:31:02 +0000 2025,no patriarki di keluarga gua since my mother &...,dhislhc
1,Sat Mar 29 14:22:50 +0000 2025,Wlgt bener-bener pelajaran hidup banget.. mung...,tenniewinibiti
2,Wed Mar 12 15:39:25 +0000 2025,GUE UDAH NGELAKUIN INI SINCE PERBAIKOTAN ITU P...,wdnrvzy
3,Fri Mar 07 16:22:37 +0000 2025,@meongpundung well i've been the novel reader ...,plaveshinki
4,Tue Feb 25 03:59:27 +0000 2025,Perempuan jongos patriarki itu level menjijikk...,aliciasoetopo
...,...,...,...
14201,Sat Mar 23 16:53:56 +0000 2024,@sosmedkeras Betina,AkbarAgasaki
14202,Sat Mar 23 13:33:33 +0000 2024,@KatzNada3 Hihihihihi ejke betina ye Ketikan l...,Atheis_Bergitar
14203,Sat Mar 23 11:39:00 +0000 2024,@LiatAjaX Dasar betina rahim anget ‍️,AkbarAgasaki
14204,Fri Mar 22 21:00:58 +0000 2024,@RaisaTjokrodnta Junjungan lo kan sdh menang t...,Ibnu07338018


In [10]:
def clean_text(s):
    s = str(s).lower()
    s = re.sub(r'http\S+|@\w+', ' ', s)
    s = re.sub(r'[^a-z0-9\s]', ' ', s)
    return re.sub(r'\s+', ' ', s).strip()

df['cleaned'] = df['full_text'].apply(clean_text)

In [11]:
for cat, kws in keywords.items():
    pattern = r'\b(?:' + '|'.join(map(re.escape, kws)) + r')\b'
    df[cat] = df['cleaned'].apply(lambda txt: int(bool(re.search(pattern, txt))))

In [12]:
if 'label_single' in df.columns:
    df.drop('label_single', axis=1, inplace=True)

In [13]:
df['label_UK'] = df.apply(
    lambda row: [cat for cat in keywords.keys() if row[cat]==1],
    axis=1
)

In [14]:
os.makedirs('data', exist_ok=True)
df.to_csv('data/uk_multilabel.csv', index=False)

In [15]:
os.makedirs('corpus/words', exist_ok=True)
os.makedirs('corpus/sents', exist_ok=True)

In [16]:
for cat, kws in keywords.items():
    # kata
    words = set()
    for txt in df.loc[df[cat]==1, 'cleaned']:
        words.update(txt.split())
    pd.DataFrame({'word': sorted(words)}) \
      .to_csv(f'corpus/words/{cat}.csv', index=False)

In [17]:
    rows = []
    pat = r'\b(?:' + '|'.join(map(re.escape, kws)) + r')\b'
    for full in df['full_text'].dropna():
        for sent in re.split(r'(?<=[\.\?\!])\s+', full):
            if re.search(pat, sent, flags=re.IGNORECASE):
                rows.append({'sentence': sent.strip()})
    pd.DataFrame(rows) \
      .to_csv(f'corpus/sents/{cat}.csv', index=False)

In [20]:
print(df[['full_text','cleaned'] + list(keywords.keys()) + ['label_single']].head())

KeyError: "['label_single'] not in index"